In [1]:
!pip install pyserial

In [ ]:
import serial
import time
from http.server import BaseHTTPRequestHandler, HTTPServer

# Set up serial communication (change COM port if needed)
arduino = serial.Serial('COM7', 9600)  # Replace 'COM7' with your port
time.sleep(2)  # Wait for Arduino to initialize

# Create a web server to display the temperature
class RequestHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        if self.path == "/" or self.path == "/favicon.ico":
            # Ignore favicon.ico requests
            if self.path == "/favicon.ico":
                self.send_response(204)
                self.end_headers()
                return

            # Read temperature data from the Arduino
            if arduino.in_waiting > 0:
                temp = arduino.readline().decode('utf-8').strip()
                
                # Send the HTML response to the browser
                self.send_response(200)
                self.send_header('Content-type', 'text/html')
                self.end_headers()
                self.wfile.write(f"""
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1.0"/>
  <title>Arduino Thermometer</title>
  <style>
    * {{
      margin: 0;
      padding: 0;
      box-sizing: border-box;
    }}

    body {{
      font-family: 'Arial', sans-serif;
      line-height: 1.6;
      color: #333;
      background-color: #fafafa;
      margin: 0;
      padding: 0;
    }}

    header {{
      display: none;
    }}

    .hero {{
      background: linear-gradient(135deg, #FDCB82, #F16D74);
      height: 100vh;
      display: flex;
      justify-content: center;
      align-items: center;
      color: white;
      text-align: center;
      padding: 0 20px;
    }}

    .hero-content h1 {{
      font-size: 3rem;
      margin-bottom: 20px;
      animation: fadeIn 1s ease-out;
    }}

    .hero-content p {{
      font-size: 1.5rem;
      margin-bottom: 30px;
      animation: fadeIn 2s ease-out;
    }}

    .hero .cta-btn {{
      background-color: #FF6F61;
      color: white;
      font-size: 1.2rem;
      padding: 12px 30px;
      text-decoration: none;
      border-radius: 5px;
      transition: background-color 0.3s;
      animation: fadeIn 3s ease-out;
    }}

    .hero .cta-btn:hover {{
      background-color: #FF9A8B;
    }}
  </style>
</head>
<body>

  <section class="hero">
    <div class="hero-content">
      <h1>🌡 Real-Time Arduino Temperature Monitor</h1>
      <p>Monitor and control the temperature with Arduino and LM35 sensor</p>
      <a href="introduction.html" class="cta-btn">Learn More</a> <!-- Updated href -->
    </div>
  </section>

</body>
</html>""".encode('utf-8'))
            else:
                # If there's no data from Arduino
                self.send_response(500)
                self.send_header('Content-type', 'text/html')
                self.end_headers()
                self.wfile.write("<html><body><h1>Waiting for Arduino data...</h1></body></html>".encode('utf-8'))

# Start the server on port 8080
server_address = ('', 8080)
httpd = HTTPServer(server_address, RequestHandler)
print("Server running on http://localhost:8080")
httpd.serve_forever()

Server running on http://localhost:8080
